In [1]:
import pandas as pd
import numpy as np
import sklearn.metrics
import sklearn.model_selection
import time

In [2]:
import h2o
from h2o.automl import H2OAutoML

In [3]:
def compute(x_path, y_path,comp_time):
    X = pd.read_csv(x_path)
    y = pd.read_csv(y_path)
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=7)
    del X,y
    df_train = pd.concat([X_train, y_train], axis=1)
    
    hf_train = h2o.H2OFrame(df_train)
    
    t0 = time.time()
    
    x_names = hf_train.columns
    y_names = "y"
    x_names.remove(y_names)
    
    aml = H2OAutoML(max_runtime_secs=comp_time, exclude_algos=['DeepLearning'], seed = 1, verbosity="NULL")
    aml.train(x=x_names, y=y_names, training_frame=hf_train)
    
    t1 = time.time()
    print("training time cost:",t1-t0)
    
    # View the AutoML Leaderboard
    lb = aml.leaderboard
    print(lb.head(10))  # Print all rows instead of default 
    
    train_predictions = aml.predict(h2o.H2OFrame(X_train))
    train_predictions = h2o.as_list(train_predictions)
    print("Train R2 score:", sklearn.metrics.r2_score(y_train, train_predictions))
    test_predictions = aml.predict(h2o.H2OFrame(X_test))
    test_predictions = h2o.as_list(test_predictions)
    print("Test R2 score:", sklearn.metrics.r2_score(y_test, test_predictions))
        
    t2 = time.time()
    print("prediction time cost",t2-t1)
    m = aml.get_best_model()
    print(pd.DataFrame(m.metalearner().coef_norm(),index=["r2"]).T.sort_values('r2',ascending = False))

In [4]:
h2o.init()
for i in [4,14]:
    for comp_time in [600, 1200, 1800, 2400, 3000, 7200, 10800]:
        x_path = 'dataset/QSAR_{}_train_x.csv'.format(i)
        y_path = 'dataset/QSAR_{}_train_y.csv'.format(i)
        compute(x_path, y_path, comp_time)
        print("now is processing dataset %s" % i)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_152-release"; OpenJDK Runtime Environment (build 1.8.0_152-release-1056-b12); OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)
  Starting server from /cache/home/hx152/.conda/envs/merck/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp_l2gm0wq
  JVM stdout: /tmp/tmp_l2gm0wq/h2o_hx152_started_from_python.out
  JVM stderr: /tmp/tmp_l2gm0wq/h2o_hx152_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.3
H2O_cluster_version_age:,1 month and 25 days
H2O_cluster_name:,H2O_from_python_hx152_gifik9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,26.67 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
training time cost: 604.1378629207611


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_1_AutoML_1_20220411_161053,0.118079,0.343626,0.118079,0.271975,0.0467364
StackedEnsemble_BestOfFamily_3_AutoML_1_20220411_161053,0.120282,0.346817,0.120282,0.269001,0.046994
StackedEnsemble_BestOfFamily_2_AutoML_1_20220411_161053,0.120501,0.347133,0.120501,0.270161,0.0470502
StackedEnsemble_AllModels_1_AutoML_1_20220411_161053,0.123588,0.351551,0.123588,0.271056,0.0477576
StackedEnsemble_AllModels_2_AutoML_1_20220411_161053,0.123997,0.352132,0.123997,0.272216,0.0478137
StackedEnsemble_AllModels_3_AutoML_1_20220411_161053,0.124253,0.352496,0.124253,0.275833,0.0479388
XRT_1_AutoML_1_20220411_161053,0.129519,0.359887,0.129519,0.280363,0.048797
XGBoost_1_AutoML_1_20220411_161053,0.129943,0.360476,0.129943,0.276227,0.049229
GBM_4_AutoML_1_20220411_161053,0.134592,0.366868,0.134592,0.272115,0.0495804
GBM_grid_1_AutoML_1_20220411_161053_model_5,0.136055,0.368856,0.136055,0.273014,0.0498517



Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Train R2 score: 0.8073969092120667
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Test R2 score: 0.5414065446456875
prediction time cost 4.867479085922241
                                          r2
Intercept                           6.361186
GLM_1_AutoML_1_20220411_161053      0.221940
XGBoost_1_AutoML_1_20220411_161053  0.201598
GBM_1_AutoML_1_20220411_161053      0.000000
now is processing dataset 4
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |██████████████████████████████████████████████████████████| (done) 100%
training time cost: 1201.2041547298431


...


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Train R2 score: 0.9241426105507835
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Test R2 score: 0.45775124405567436
prediction time cost 13.404942512512207
                                                       r2
Intercept                                        0.355863
XGBoost_grid_1_AutoML_12_20220412_02827_model_2  0.064831
XGBoost_grid_1_AutoML_12_20220412_02827_model_1  0.055590
GBM_4_AutoML_12_20220412_02827                   0.041573
XGBoost_grid_1_AutoML_12_20220412_02827_model_4  0.037467
GBM_1_AutoML_12_20220412_02827                   0.036231
XGBoost_2_AutoML_12_20220412_02827               0.035444
GLM_1_AutoML_12_20220412_02827               

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_3_AutoML_13_20220412_11901,0.111917,0.33454,0.111917,0.249515,nan
StackedEnsemble_Best1000_1_AutoML_13_20220412_11901,0.112153,0.334893,0.112153,0.250101,nan
StackedEnsemble_AllModels_6_AutoML_13_20220412_11901,0.112292,0.335099,0.112292,0.250434,nan
StackedEnsemble_BestOfFamily_7_AutoML_13_20220412_11901,0.113849,0.337415,0.113849,0.254044,nan
StackedEnsemble_BestOfFamily_4_AutoML_13_20220412_11901,0.113935,0.337542,0.113935,0.253929,nan
StackedEnsemble_AllModels_5_AutoML_13_20220412_11901,0.114021,0.33767,0.114021,0.252322,nan
StackedEnsemble_AllModels_2_AutoML_13_20220412_11901,0.114628,0.338567,0.114628,0.25431,nan
StackedEnsemble_AllModels_1_AutoML_13_20220412_11901,0.115157,0.339347,0.115157,0.254836,nan
StackedEnsemble_BestOfFamily_2_AutoML_13_20220412_11901,0.115202,0.339414,0.115202,0.254571,nan
StackedEnsemble_BestOfFamily_3_AutoML_13_20220412_11901,0.115707,0.340158,0.115707,0.255379,nan



Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Train R2 score: 0.9445186667878813
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Test R2 score: 0.46142021904282027
prediction time cost 15.411638498306274
                                                        r2
Intercept                                         0.355863
GBM_grid_1_AutoML_13_20220412_11901_model_1       0.056638
XGBoost_grid_1_AutoML_13_20220412_11901_model_28  0.028639
XGBoost_grid_1_AutoML_13_20220412_11901_model_13  0.028253
XGBoost_grid_1_AutoML_13_20220412_11901_model_6   0.026312
XGBoost_grid_1_AutoML_13_20220412_11901_model_1   0.026302
XGBoost_grid_1_AutoML_13_20220412_11901_model_24  0.025761
XGBoost_grid_1_AutoML_13_20220412_119

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_3_AutoML_14_20220412_31908,0.11147,0.333872,0.11147,0.249287,nan
StackedEnsemble_Best1000_1_AutoML_14_20220412_31908,0.111826,0.334404,0.111826,0.25001,nan
StackedEnsemble_AllModels_6_AutoML_14_20220412_31908,0.111842,0.334428,0.111842,0.249926,nan
StackedEnsemble_BestOfFamily_7_AutoML_14_20220412_31908,0.113817,0.337368,0.113817,0.253779,nan
StackedEnsemble_BestOfFamily_4_AutoML_14_20220412_31908,0.114005,0.337647,0.114005,0.253846,nan
StackedEnsemble_AllModels_2_AutoML_14_20220412_31908,0.114699,0.338673,0.114699,0.254292,nan
StackedEnsemble_BestOfFamily_2_AutoML_14_20220412_31908,0.115169,0.339365,0.115169,0.254312,nan
StackedEnsemble_AllModels_1_AutoML_14_20220412_31908,0.115215,0.339434,0.115215,0.25484,nan
StackedEnsemble_BestOfFamily_3_AutoML_14_20220412_31908,0.115603,0.340004,0.115603,0.255206,nan
StackedEnsemble_AllModels_5_AutoML_14_20220412_31908,0.115827,0.340334,0.115827,0.255137,nan



Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Train R2 score: 0.9463481692022883
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Test R2 score: 0.4660086330613551
prediction time cost 16.8273286819458
                                                        r2
Intercept                                         0.355863
XGBoost_grid_1_AutoML_14_20220412_31908_model_62  0.042983
GBM_grid_1_AutoML_14_20220412_31908_model_1       0.041997
XGBoost_grid_1_AutoML_14_20220412_31908_model_50  0.040312
XGBoost_grid_1_AutoML_14_20220412_31908_model_37  0.031368
...                                                    ...
XGBoost_grid_1_AutoML_14_20220412_31908_model_32  0.000000
XGBoost_grid_1_AutoML_14_20220412_31908_